Import libraries and use Beautiful Soup to get table from web page

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
 
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [2]:
table = soup.find('table', {'class':'wikitable sortable'})

In [3]:
links = table.findAll('a')

Append values from the table into three arrays and create data frame with these arrays

In [4]:
Postal_codes = []
Boroughs = []
Neighbourhoodsraw = []
Neighbourhoods = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        Postal_codes.append(cells[0].find(text=True))
        Boroughs.append(cells[1].find(text=True))
        Neighbourhoodsraw.append(cells[2].find(text=True))

df = pd.DataFrame()
df['Postal Codes'] = Postal_codes
df['Borough'] = Boroughs
df['Neighborhood'] = Neighbourhoodsraw

Remove rows where Borough is not assigned

In [5]:
df = df[~df.Borough.str.contains('Not assigned')]

For values in not assigned in Neighbourhood column, assign the same Borough value.
Then remove \n from strings 

In [6]:
for i, j in df.iterrows():
    if j['Neighborhood'] == 'Not assigned\n':
        j['Neighborhood'] = j['Borough']
        
for i in df['Neighborhood']:
    if '\n' in i:
        i = i[:-1]
        Neighbourhoods.append(i)
    else:
        Neighbourhoods.append(i)
        

In [7]:
df['Neighborhood'] = Neighbourhoods

Group all neighbourhoods with same postal code

In [8]:
df = df.groupby(['Postal Codes', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()


In [9]:
df

,Postal Codes,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [10]:
df.shape

(103, 3)

Use Geocoder to get longitude/latitude
Since both dataframes are grouped by Postal Code, we can copy the two wanted columns

In [11]:
locations = pd.read_csv('Geospatial_Coordinates.csv') 

In [12]:
locations

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [13]:
df['Latitude'] = locations['Latitude']
df['Longitude'] = locations['Longitude']

In [14]:
df

,Postal Codes,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


# Mapping the Neighbourhoods

Import tools neccessary to produce map and clusters

In [15]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

Create a map, marking the different postal coded areas

In [16]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitudes = location.latitude
longitudes = location.longitude

In [18]:
tor_map = folium.Map(location =[latitudes, longitudes], zoom_start = 10)

for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)  
tor_map

Let us explore Scarborough
&
Define four squares ID

In [19]:
CLIENT_ID = 'FOVA0ZMSD3ONHLRE5AWRE0QEMTHWKSUXDLEGRO53LS0TPYBP'
CLIENT_SECRET = 'B1W23OBLYPZWKA4TSNZJH45ZCNHAI2XXVAVB2PQYXTY4S3SW'
VERSION = '20190914'

In [20]:
scarborough_data = df[df['Borough']=='Scarborough'].reset_index(drop = True)
LIMIT = 20

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [23]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [24]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest, Cliffside, Scarborough Village West",4,4,4,4,4,4
"Dorset Park, Scarborough Town Centre, Wexford Heights",9,9,9,9,9,9
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


In [25]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3      Sandwich Place   0.2
4  Chinese Restaurant   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                       Park  0.33
1                 Playground  0.33
2                Coffee Shop  0.33
3        American Restaurant  0.00
4  Latin American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0               Bakery  0.14
1                 Bank  0.14
2  Fried Chicken Joint  0.14
3      Thai Restaurant  0.14
4   Athletics & Sports  0.14


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2  Intersec

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,College Stadium,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Coffee Shop,Playground,Park,Vietnamese Restaurant,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Department Store,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Bus Station,Soccer Field,Metro Station,Park,Fast Food Restaurant,Discount Store,Electronics Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pharmacy,Pizza Place,Fast Food Restaurant,Italian Restaurant,Noodle House,Fried Chicken Joint,Chinese Restaurant,Shopping Mall,Bank,Thai Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",Motel,American Restaurant,Movie Theater,Hobby Shop,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Gaming Cafe,Latin American Restaurant,Light Rail Station,Furniture / Home Store,Chinese Restaurant,Vietnamese Restaurant,Bus Line,Electronics Store
8,"East Birchmount Park, Ionview, Kennedy Park",Coffee Shop,Bus Station,Discount Store,Department Store,Convenience Store,Hobby Shop,Breakfast Spot,Athletics & Sports,Hakka Restaurant,Grocery Store
9,"Guildwood, Morningside, West Hill",Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Mexican Restaurant,College Stadium,General Entertainment,Gaming Cafe


# k-Cluster

In [28]:
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

kmeans.labels_[0:10]

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Top venues for each postal code area shown, with their corresponding cluster label to be mapped

In [44]:
scarborough_merged = scarborough_data
scarborough_merged
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
scarborough_merged = scarborough_merged.dropna(axis = 0).astype({"Cluster Labels": int})
scarborough_merged.head(10)

,Postal Codes,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Mexican Restaurant,College Stadium,General Entertainment,Gaming Cafe
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Department Store,Grocery Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,1,Coffee Shop,Bus Station,Discount Store,Department Store,Convenience Store,Hobby Shop,Breakfast Spot,Athletics & Sports,Hakka Restaurant,Grocery Store
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Intersection,Bus Station,Soccer Field,Metro Station,Park,Fast Food Restaurant,Discount Store,Electronics Store
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,1,Motel,American Restaurant,Movie Theater,Hobby Shop,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint


In [43]:
map_clusters = folium.Map(location=[latitudes, longitudes], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Note the majority of the scarborough venues in purple clusters. For the other 4 "clusters", only 1 area is found in them. 